In [1]:
from app.Transformer_Classifier import Transformer_Classifier 
from app.Data_Loader import Data_Loader
from app.TextRank_Extractor import TextRank_Extractor
from app.Keyword_Classifier import Keyword_Classifier
from app.common.MySQLUtility import MySQLUtility
import os 

domains = ['esg', 'liabilities' ] #'liabilities', 'esg'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = './store/genuine-wording-key.json'

DB_HOST = '34.170.168.203'
DB_USER = 'root'
DB_PASSWORD = 'nu123456'
DB_NAME = 'lca_db'

class Data_ETL_Pipeline(object):
    dbutil = None
    data_load = None
    textrank = None 
    key_classifier = None
    class_service = None

    def __init__(self):
        self.dbutil = MySQLUtility(DB_HOST, DB_USER, DB_PASSWORD, DB_NAME)
        self.data_load = Data_Loader(self.dbutil)
        self.textrank = TextRank_Extractor(self.dbutil)
        self.key_classifier = Keyword_Classifier(self.dbutil)
        self.class_service = Transformer_Classifier(self.dbutil)
        pass    

    def create_dataset(self):
        print("dbutil.db_cleanup():")
        self.dbutil.clean_db()
        print("dbutil.create_database():")
        self.dbutil.create_database() 

    def load_seed_training_data(self):
        print("data_load.import_seed_data_batch():")
        self.data_load.import_seed_data_batch()

        for domain in domains:
            print("textrank.extract_keyword_seed_data():" + domain)
            self.textrank.extract_keyword_seed_data(domain) 

            print("textrank.load_seed_to_training_data_batch():" + domain)
            self.data_load.load_seed_to_training_data_batch(domain) 
    
    def load_contract_data(self):
        for domain in domains:
            print("self.data_load.import_reports_contract_data()" + domain)
            self.data_load.import_reports_contract_data(domain)

    def process_keyword_model(self):
        for domain in domains:
            print("key_classifier.prepare_training_data():" + domain)
            self.key_classifier.prepare_training_data(domain)

            print("key_classifier.train_model():" + domain)
            self.key_classifier.train_model(domain)

            print("key_classifier.evaluate_model():" + domain)
            self.key_classifier.evaluate_model(domain)

            print("key_classifier.process_contract_data():" + domain)
            self.key_classifier.process_contract_data(domain)

    def process_transformer_model(self):
        for domain in domains:
            print("class_service.training():" + domain)
            self.class_service.training(domain)    

            print("class_service.process_contract_training_data_eval():" + domain)
            self.class_service.process_contract_training_data_eval(domain)

    def evaluate_results(self):
        for domain in domains:
            print ("key_classifier.Keyword Classifier Accuracy: " + domain)
            self.key_classifier.evaluate_model(domain) 
            
            print ("class_service.Transformer Classifier Accuracy: " + domain)
            self.class_service.evalute_model(domain)

if __name__ == '__main__': 
    dbloader = Data_ETL_Pipeline()
    #dbloader.create_dataset()
    #dbloader.load_seed_training_data() 
    #dbloader.load_contract_data()
    #dbloader.process_keyword_model()
    dbloader.process_transformer_model()
    dbloader.evaluate_results()


/Users/saurabhkaushik/Workspace/lca-ai-services/env/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


class_service.training():liabilities
DB Pool Created.
Query:  SELECT * from training_data where domain='liabilities';
SELECT * from training_data where domain='liabilities';
Query:  SELECT * from training_data where domain='liabilities';
SELECT * from training_data where domain='liabilities';


ERROR:root:Traceback (most recent call last):
  File "/Users/saurabhkaushik/Workspace/lca-ai-services/app/Transformer_Classifier.py", line 81, in training
    train_hg, valid_hg = self.prepare_train_dataset(domain)
  File "/Users/saurabhkaushik/Workspace/lca-ai-services/app/Transformer_Classifier.py", line 68, in prepare_train_dataset
    random_state=2022
  File "/Users/saurabhkaushik/Workspace/lca-ai-services/env/lib/python3.7/site-packages/sklearn/model_selection/_split.py", line 2421, in train_test_split
    n_samples, test_size, train_size, default_test_size=0.25
  File "/Users/saurabhkaushik/Workspace/lca-ai-services/env/lib/python3.7/site-packages/sklearn/model_selection/_split.py", line 2101, in _validate_shuffle_split
    "aforementioned parameters.".format(n_samples, test_size, train_size)
ValueError: With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.



class_service.process_contract_training_data_eval():liabilities
Query:  SELECT * from training_data where domain='liabilities';
SELECT * from training_data where domain='liabilities';
0
None
-1 record(s) affected
key_classifier.Keyword Classifier Accuracy: liabilities
SELECT * from seed_data where domain='liabilities';


ERROR:root:Traceback (most recent call last):
  File "/Users/saurabhkaushik/Workspace/lca-ai-services/app/Keyword_Classifier.py", line 66, in evaluate_model
    predicted = self.text_clf.predict(self.trainDF['text'])
  File "/Users/saurabhkaushik/Workspace/lca-ai-services/env/lib/python3.7/site-packages/sklearn/utils/metaestimators.py", line 113, in <lambda>
    out = lambda *args, **kwargs: self.fn(obj, *args, **kwargs)  # noqa
  File "/Users/saurabhkaushik/Workspace/lca-ai-services/env/lib/python3.7/site-packages/sklearn/pipeline.py", line 469, in predict
    Xt = transform.transform(Xt)
  File "/Users/saurabhkaushik/Workspace/lca-ai-services/env/lib/python3.7/site-packages/sklearn/feature_extraction/text.py", line 1661, in transform
    X, accept_sparse="csr", dtype=FLOAT_DTYPES, copy=copy, reset=False
  File "/Users/saurabhkaushik/Workspace/lca-ai-services/env/lib/python3.7/site-packages/sklearn/base.py", line 566, in _validate_data
    X = check_array(X, **check_params)
  File "/U

Series([], Name: label, dtype: float64)
class_service.Transformer Classifier Accuracy: liabilities
Query:  SELECT * from training_data where domain='liabilities';
SELECT * from training_data where domain='liabilities';



In [1]:
from app.common.GCP_Storage import GCP_Storage

import os 
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = './store/genuine-wording-key.json'

domains = ['liabilities', 'esg']
loader = GCP_Storage(domains)

#loader.setup_bucket()
loader.upload_models()
#loader.download_models()
#loader.download_seed_data()
#loader.upload_seed_data()

/Users/saurabhkaushik/Workspace/lca-ai-services/env/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Uploading Models to GCP Bucket.
File ./model/liabilities/config.json uploaded to liabilities/config.json.
Uploaded liabilities/config.json.
File ./model/liabilities/keyword_class.joblib uploaded to liabilities/keyword_class.joblib.
Uploaded liabilities/keyword_class.joblib.
File ./model/liabilities/pytorch_model.bin uploaded to liabilities/pytorch_model.bin.
Uploaded liabilities/pytorch_model.bin.
File ./model/esg/config.json uploaded to esg/config.json.
Uploaded esg/config.json.
File ./model/esg/keyword_class.joblib uploaded to esg/keyword_class.joblib.
Uploaded esg/keyword_class.joblib.
File ./model/esg/pytorch_model.bin uploaded to esg/pytorch_model.bin.
Uploaded esg/pytorch_model.bin.
